In [ ]:
#legacy code that was used to scrape cuspajz.com before switching to tekstovi.net as main songbase
import pandas as pd
import base64
import os

from requests import get
from bs4 import BeautifulSoup
from time import sleep

## Create empty df:
column_names = ["Artist", "ArtistDiskografija", "Song_ID", "Song", "Lyrics", "Url"]
df = pd.DataFrame(columns = column_names)




In [ ]:
# Scraping cuspajz.com for song lyrics by artist (change url to change artist)

# Based on:
#    https://www.dataquest.io/blog/web-scraping-beautifulsoup/
#    https://github.com/aakashbansal/Songs-Lyrics-Web-Scraper/blob/master/Lyrics%20Scraper.ipynb

artist = 'Severina'
diskografija_artist_name = 'Severina'
# artist url
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/severina.html'

# generate random artist ID and make sure it doesnt have '_'
artist_ID = base64.b64encode(os.urandom(6)).decode('ascii')
while(artist_ID.find('_')>=0):
    artist_ID = base64.b64encode(os.urandom(6)).decode('ascii')
    
delay = 2

response = get(url)
# print(response.text[:500])
html_soup = BeautifulSoup(response.text, 'html.parser')
# print(type(html_soup))


# Get the list of songs
songs = html_soup.find_all('ul', class_='songList')



In [ ]:
for col_index, column in enumerate(songs):
    songs = column.find_all('li')

    for index, song in enumerate(songs):

        song_name = song.a.text
        song_url = song.a['href']

        url = 'https://cuspajz.com/'+song_url

        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')

        song_lyrics = html_soup.find_all('p', class_='clearfix')
        lyrics =  song_lyrics[0].get_text()
            
        song_ID = artist_ID + '_' + str(1) + '_' + str(index)  

        new_entry = pd.DataFrame([[artist, diskografija_artist_name, song_name, song_ID, lyrics, url]], columns=["Artist", "ArtistDiskografija", "Song", "Song_ID", "Lyrics", "Url"])
        df = df.append(new_entry, ignore_index=True)

        print("Lyrics successfully written to file for : " + song_name)
                      
        sleep(delay)
                      

In [ ]:
df.to_csv(f'data/{artist}_cuspajz.csv', index=False)